# DUMP the model to PM, allowing for it to run on Android Devices

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!kaggle

In [ ]:
!cp -f kaggle.json /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
%%capture
!kaggle datasets download -d kneroma/tacotrashdataset -p ../input/tacotrashdataset
!unzip ../input/tacotrashdataset/tacotrashdataset.zip -d ../input/tacotrashdataset/ 

# Import Dependencies

In [258]:
import numpy as np
import pandas as pd

Data Dir

In [259]:
class DataModule:
  base_dir = './drive/MyDrive/EARTH_Models/'
  csv_file = '../input/tacotrashdataset/meta_df.csv'
  csv_module = pd.read_csv(csv_file)
  ALL_CLASSES = sorted(csv_module.cat_name.unique())

  data_csv = '../input/tacotrashdataset/meta_df.csv' # All the Data inside of TacoTrashDataset
  # Needs to be split.
  data_df = pd.read_csv(data_csv)
  # Change the paths to the BASE DATA DIR
  BASE_DATA_DIR = '../input/tacotrashdataset/data/'
  data_df['img_file'] = BASE_DATA_DIR + data_df['img_file']

  # ---------BASIC DATA PREP--------------
  # Convert the Multiple Rows of the DF into a single Caption
  all_unique_ids = data_df.img_file.unique()
  # One-Hot Encode classes
  classes2idx = {}
  idx2classes = {}
  ALL_CLASSES = sorted(data_df.cat_name.unique())
  for idx, class_name in enumerate(ALL_CLASSES):
    idx2classes[idx] = class_name
    classes2idx[class_name] = idx
  NUM_CLASSES = len(classes2idx)
  
  PAD_BOUNDING_BOXES = -100 # Pad with < -1. This Ignores the Regression Targets.

  TARGET_DIR = './drive/MyDrive/EARTH_Models/'

Pickle the classes

In [260]:
with open(f"{DataModule.base_dir}classes.txt", 'w') as file:
  file.write(str(DataModule.ALL_CLASSES))

# Step 3: Prepare the model for direct inference
- Instead of outputting logits, output the Bounding Boxes themselves
- Jit Compile, collapsing this into a function

In [261]:
%%capture
!wget https://github.com/rwightman/efficientdet-pytorch/archive/refs/heads/master.zip
!unzip ./master.zip
!rm -f ./master.zip

In [262]:
%%capture 
!pip install pycocotools
!pip install timm
!pip install omegaconf

In [263]:
%%capture
%cd efficientdet-pytorch-master/
from effdet import get_efficientdet_config, EfficientDet, DetBenchTrain
from effdet.efficientdet import HeadNet
import effdet
%cd ..

In [264]:
%%capture
import math
import copy
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt 

import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from sklearn.model_selection import ShuffleSplit
!pip install pytorch_lightning
import pytorch_lightning as pl

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [265]:
from effdet import *

# Create the Model

In [266]:
class TRAININGOBJECT(nn.Module):
  def __init__(self):
    super().__init__()    
    self.model = get_net()

In [267]:
class PyTorchLightningModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.model = self.configure_model()


    self.train_loss = 0.0
    self.train_steps = 0
    self.val_loss = 0.0
    self.val_steps = 0
  def configure_model(self):
    model = TRAININGOBJECT()
    return model 
  def configure_optimizers(self):
    optimizer = optim.AdamW(self.logger_object.training_object.model.parameters(), 
        lr = self.logger_object.training_object.lr,
        weight_decay = self.logger_object.training_object.weight_decay)
    
    lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 5, eta_min = 1e-13, verbose = True)
    return {'optimizer': optimizer, 'lr_scheduler': lr_scheduler}
  def training_step(self, batch, batch_idx):
    images, bboxes, classification = batch
    images = images.half()
    loss = train_step(images, bboxes, classification, training_object=self.logger_object.training_object)
    self.train_loss += loss.item()
    self.train_steps += 1
    return loss
  def validation_step(self, batch, batch_idx):
    images, bboxes, classification = batch
    images = images.half()
    loss = val_step(images, bboxes, classification, training_object=self.logger_object.training_object)
    self.val_loss += loss.item()
    self.val_steps += 1
  def validation_epoch_end(self, logs):
    eps = 1e-10
    self.logger_object.update_states((self.train_loss + eps) / (self.train_steps + eps), (self.val_loss + eps) / (self.val_steps + eps))
    self.train_loss = 0.
    self.train_steps = 0.
    self.val_loss = 0.
    self.val_steps = 0.0


In [268]:
IMG_SIZE = 512

In [269]:
class PyTorchLightningModelWrapper(nn.Module):
  def __init__(self, model):
    super().__init__()
    self.model = model
  def forward(self, *args, **kwargs):
    return self.model(*args, **kwargs)
def get_net():
    config = get_efficientdet_config('tf_efficientdet_d0') # Smallest Model Possible, to make it runable on CPU(Especially an Android CPU)
    net = EfficientDet(config, pretrained_backbone=False)

    effdet.config.config_utils.set_config_writeable(config) 
    config.num_classes = DataModule.NUM_CLASSES
    config.image_size = (IMG_SIZE, IMG_SIZE)
    effdet.config.config_utils.set_config_readonly(config)
    net.class_net = HeadNet(config, num_outputs=config.num_classes)
    model = DetBenchTrain(net, config)
    return PyTorchLightningModelWrapper(model)


In [270]:
class InferenceModel(nn.Module):
  def __init__(self, prev_model):
    super().__init__()
    self.prev_model = prev_model 
    self.model = PyTorchLightningModel()
    self.model.model.model.model.load_state_dict(torch.load(self.prev_model, map_location = device))
    self.model = DetBenchPredict(self.model.model.model.model.model).to(device)
  def forward(self, x):
    self.eval()
    B = x.shape[0]
    with torch.no_grad():
      x = x.to(device)
      predictions =  self.model(x, img_info = {'img_scale': torch.tensor([1.0] * B, dtype=torch.float).to(device), 
      'img_size':  torch.tensor([x[0].shape[-2:]] * B, 
      dtype=torch.float).to(device)})
      # Hard NMS
      idx = torchvision.ops.nms(predictions[0, :, :4], predictions[0, :, 4], iou_threshold = 0.5)
      predictions = predictions[0, idx]
      return predictions
    

In [271]:
model = InferenceModel(DataModule.base_dir + 'final.pth')

# Serialize the model
- Model Output: Tensor(N,6): (x1, y1, x2, y2, obj, cls)

In [272]:
from torch.utils.mobile_optimizer import optimize_for_mobile

In [273]:
# Trace the Model, send it to torch.mobile
example = torch.rand(1, 3, IMG_SIZE, IMG_SIZE)
traced_script_module = torch.jit.trace(model, example)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("./model.ptl")


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, 